In [47]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
#from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [48]:
#DATA sets
#shapefiles
unimodal_shapefile='rheas/gadm36_TZA_LongRain.shp'
bimodal_shapefile='rheas/gadm36_TZA_2_Bimodal.shp'
country_shapefile='rheas/gadm36_TZA_0.shp'
unimodal_csv='rheas/tanzania_tamsat_25km_districts_unimod_dssatTable_2021_2023_100kg.csv'
bimodal_csv='rheas/tanzania_tamsat_25km_districts_bimod_dssatTable_2000_2023_100kg.csv'


In [49]:
def format_df(filename):
    df = pd.read_csv(filename)
    df['harvest'] = pd.to_datetime(df['harvest'], format='%Y/%m/%d')
    df['planting'] = pd.to_datetime(df['planting'], format='%Y/%m/%d')
    df['date'] = df['harvest'].dt.strftime("%Y")
    df['gwad']=df['gwad']/1000
    df = df.rename(columns={df.columns[1]: "Region"})
    #df['Region'] = df['Region'].str.upper()
    return df

In [50]:
unimodal_data = format_df(unimodal_csv)

In [51]:
print(unimodal_data.head(3))

      id Region   planting    harvest   wsgd   gwad  date
0  21665  Nyasa 2023-02-18 2023-06-22  0.166  3.063  2023
1  21666  Nyasa 2023-01-22 2023-06-02  0.000  3.297  2023
2  21667  Nyasa 2023-02-16 2023-06-22  0.311  3.194  2023


In [52]:
# Unimodal season:  October--August (LR)
lrStart = [9, 10, 11, 12, 1,2, 3, 4]
lrEnd = [2, 3, 4, 5, 6, 7, 8]
def LR_metrics(lr, lrStart, lrEnd, season):
    lr.loc[(pd.to_datetime(lr['planting']).dt.month.isin(lrStart)) & (pd.to_datetime(lr['harvest']).dt.month.isin(lrEnd)), 'Season'] = season
    lr = lr.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})
    lr=lr[lr['Season']=='LR']

    return lr
rh_lr = LR_metrics(unimodal_data, lrStart, lrEnd, "LR")  
#print(rh_lr.head())

In [53]:
# bimodal season:  October--August (OND)
bimodal_data = format_df(bimodal_csv)
ondStart = [9, 10, 11]
ondEnd = [1, 2, 3]
def OND_metrics(ond, ondStart, ondEnd, season):
    ond.loc[(pd.to_datetime(ond['planting']).dt.month.isin(ondStart)) & (pd.to_datetime(ond['harvest']).dt.month.isin(ondEnd)), 'Season'] = season
    ond = ond.groupby(['Region', 'date', 'Season'], as_index=False).agg({'wsgd': 'mean'})

    return ond

rh_ond = OND_metrics(bimodal_data, ondStart, ondEnd, "OND")

print(rh_ond.head())

   Region  date Season      wsgd
0       1  2000    OND  0.054800
1       1  2001    OND  0.102425
2       1  2002    OND  0.000000
3       1  2003    OND  0.440075
4       1  2004    OND  0.139525


In [55]:
# Read shapefiles and merge data
unimodal_shapefile = gpd.read_file("rheas/gadm36_TZA_2_LongRain.shp")
country_shapefile = gpd.read_file("rheas/gadm36_TZA_0.shp")
unimodal_shapefile.rename(columns={'NAME_2':'Region'}, inplace=True)
unimodal_shapefile = unimodal_shapefile[['Region', 'geometry']]
rh_lr = rh_lr[['Region','Season','gwad']]
unimodal_merged_data = rh_lr.merge(unimodal_shapefile, on="Region", how='left')
unimodal_merged_data.head()

,Region,Season,gwad,geometry
0,Bahi,LR,3.783650,"POLYGON ((35.71801 -6.62413, 35.71855 -6.62543..."
1,Bahi,LR,3.123037,"POLYGON ((35.71801 -6.62413, 35.71855 -6.62543..."
2,Buhigwe,LR,3.725214,"POLYGON ((29.80319 -4.84030, 29.80203 -4.83901..."
3,Buhigwe,LR,2.691574,"POLYGON ((29.80319 -4.84030, 29.80203 -4.83901..."
4,Chamwino,LR,3.743050,"POLYGON ((35.59023 -7.01197, 35.58959 -7.01134..."


In [57]:
# Reproject to the same projection
CRS = unimodal_shapefile.crs
country_shapefile= country_shapefile.to_crs(crs=CRS)

In [58]:
# convert to geojson
#unimodal_merged_data.to_file('unimodal_merged.geojson', driver='GeoJSON')
#geojson_data =unimodal_merged_data.__geo_interface__
#geo_source = unimodal_merged_data.to_json()
#country_geo_source = country_shapefile.to_json()

geo_source = GeoJSONDataSource(geojson=unimodal_merged_data.to_json())
#country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())
#country_geo_source

OverflowError: Maximum recursion level reached

In [81]:
color_mapper = LinearColorMapper(palette=Viridis256, low=min(unimodal_merged_data[gwad]), high=max(merged_data[gwad]))

NameError: name 'gwad' is not defined

In [ ]:
p1 = figure(title=Yield Data by District,
           plot_width=650, plot_height=600)

p1.multi_line(xs,ys, source=country_geo_source,
            line_color=black, line_width=0.5)

p1.patches(xs,ys, source=geo_source,
          fill_color={field: gwad, transform: color_mapper},
          fill_alpha=0.7, line_color=black, line_width=0.5)

hover = HoverTool(tooltips=[(District, @NAME_2), (gwad, @gwad)])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation=vertical)
p1.add_layout(color_bar, right)

In [ ]:
color_mapper1 = LinearColorMapper(palette=Viridis256, low=min(merged_data[wsgd]), high=max(merged_data[wsgd]))

p2 = figure(title=Yield Data by District,
           plot_width=650, plot_height=600)

p2.multi_line(xs,ys, source=country_geo_source,
            line_color=black, line_width=0.5)

p2.patches(xs,ys, source=geo_source,
          fill_color={field: wsgd, transform: color_mapper1},
          fill_alpha=0.7, line_color=black, line_width=0.5)

hover = HoverTool(tooltips=[(District, @NAME_2), (wsgd, @wsgd)])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation=vertical)
p2.add_layout(color_bar, right)

In [ ]:
from bokeh.layouts import column
layout = column(p1, p2)
#show(layout)

In [ ]:
from bokeh.io import output_file, show
output_file(plot.html)
show(layout)

In [ ]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename=yield_map.html, title=Yield Data Map)